In [1]:
!pip install google-cloud-run google-cloud-build flask gunicorn Pillow --quiet


In [6]:

%%writefile app.py
from flask import Flask, request, send_file
from PIL import Image, ImageDraw, ImageFont
import io

app = Flask(__name__)

@app.route('/')
def home():
    return "¡Bienvenido a Cloud Run Guay! Prueba /imagen?texto=hola"

@app.route('/imagen')
def generar_imagen():
    texto = request.args.get('texto', 'Hola Mundo!')
    # Crear imagen
    img = Image.new('RGB', (500, 200), color=(73, 109, 137))
    d = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except:
        font = ImageFont.load_default()
    d.text((50, 70), texto, fill=(255, 255, 0), font=font)
    
    # Guardar en memoria
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    buf.seek(0)
    return send_file(buf, mimetype='image/png')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


Writing app.py


In [4]:
%%writefile Dockerfile
# Usamos imagen de Python ligera
FROM python:3.11-slim

# Directorio de trabajo
WORKDIR /app

# Copiar archivos
COPY app.py requirements.txt ./

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Puerto expuesto
EXPOSE 8080

# Comando para correr la app
CMD ["gunicorn", "-b", "0.0.0.0:8080", "app:app"]


Overwriting Dockerfile


In [7]:
%%writefile requirements.txt
Flask==2.3.2
Pillow==10.0.0
gunicorn==21.2.0


Overwriting requirements.txt


In [8]:
PROJECT_ID = "acecloud-sel-1"  # Cambia esto por tu proyecto
IMAGE_NAME = "cloudrun-createimage"

# Construir imagen
!gcloud builds submit --tag gcr.io/{PROJECT_ID}/{IMAGE_NAME} .


Creating temporary tarball archive of 4 file(s) totalling 4.6 KiB before compression.
Uploading tarball of [.] to [gs://acecloud-sel-1_cloudbuild/source/1767032018.14965-ced0bbe0307944219e384e529ddd77a9.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/acecloud-sel-1/locations/global/builds/71c1a505-9dc8-4cb6-b471-10cb9d30d45a].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/71c1a505-9dc8-4cb6-b471-10cb9d30d45a?project=157846830754 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "71c1a505-9dc8-4cb6-b471-10cb9d30d45a"

FETCHSOURCE
Fetching storage object: gs://acecloud-sel-1_cloudbuild/source/1767032018.14965-ced0bbe0307944219e384e529ddd77a9.tgz#1767032019411802
Copying gs://acecloud-sel-1_cloudbuild/source/1767032018.14965-ced0bbe0307944219e384e529ddd77a9.tgz#1767032019411802...
/ [1 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 1 objects/1.8 

In [ ]:

!gcloud run deploy {IMAGE_NAME} \
    --image gcr.io/{PROJECT_ID}/{IMAGE_NAME} \
    --platform managed \
    --region us-central1 \
    --allow-unauthenticated
